using Revise

In [1]:
using TopOpt, LinearAlgebra, StatsFuns
using StatsFuns: logsumexp

E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force
rmin = 3.0

3.0

### Define the problem

In [2]:
problem = PointLoadCantilever(Val{:Linear}, (160, 40), (1.0, 1.0), E, v, f)

### Parameter settings

In [3]:
V = 0.5 # volume fraction
xmin = 0.0001 # minimum density
steps = 40 # maximum number of penalty steps, delta_p0 = 0.1

40

### Continuation SIMP

In [4]:
x0 = fill(0.5, 160 * 40) # initial design
x = copy(x0)
for p in [1.0, 2.0, 3.0]
    global penalty, stress, filter, result, stress, x
    penalty = TopOpt.PowerPenalty(p)
    solver = FEASolver(Direct, problem; xmin=xmin, penalty=penalty)
    stress = TopOpt.von_mises_stress_function(solver)
    filter = DensityFilter(solver; rmin=rmin)
    volfrac = TopOpt.Volume(solver)

    obj = x -> volfrac(filter(PseudoDensities(x))) - V
    thr = 150 # stress threshold
    constr = x -> begin
        s = stress(filter(PseudoDensities(x)))
        return (s .- thr) / length(s)
    end
    alg = PercivalAlg()
    options = PercivalOptions()
    optimizer = Optimizer(obj, constr, x, alg; options=options)
    simp = SIMP(optimizer, solver, p)
    result = simp(x)
    x = result.topology
end

maximum(stress(filter(PseudoDensities(x0))))
maximum(stress(filter(PseudoDensities(x))))

[ Info:   iter        fx    normgp    normcx         μ     normy    sumc     inner_status        iter_type  
[ Info:      0  -5.5e-05   8.0e+01   1.3e+00   1.0e+01   8.0e+01       5
[ Info:      1  -5.5e-05   8.0e+01   1.3e+00   1.0e+02   8.0e+01       9      first_order         update_μ
[ Info:      2  -3.9e-04   8.0e+01   8.0e-01   1.0e+03   8.0e+01      22      first_order         update_μ
[ Info:      3  -2.3e-04   1.3e-02   8.0e-02   1.0e+03   4.3e-03      33      first_order         update_y
[ Info:      4  -1.3e-01   2.2e-01   3.8e-04   1.0e+03   3.9e-01     598         max_eval         update_y
[ Info:      5  -1.3e-01   2.2e-01   3.8e-04   1.0e+04   3.9e-01     602         max_eval         update_μ
[ Info:      6  -1.3e-01   2.0e+00   3.8e-04   1.0e+04   4.2e+00     606         max_eval         update_y
[ Info:      7  -1.3e-01   2.0e+00   3.8e-04   1.0e+05   4.2e+00     610         max_eval         update_μ
[ Info:      8  -1.3e-01   2.0e+01   3.8e-04   1.0e+05   4.3e+01     

153.12334381579024

### (Optional) Visualize the result using Makie.jl
Need to run `using Pkg; Pkg.add(Makie)` first
```julia
using Makie
using TopOpt.TopOptProblems.Visualization: visualize
fig = visualize(
   problem; topology = penalty.(filter(result.topology)), default_exagg_scale = 0.07,
   scale_range = 10.0, vector_linewidth = 3, vector_arrowsize = 0.5,
)
Makie.display(fig)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*